In [15]:
import os

In [16]:
DIR = "/Users/damoncrockett/Desktop/Leo/Images/"

In [17]:
folders = []
filenames = []

In [18]:
for root, dirs, files, in os.walk(DIR):
    for file in files:
        folders.append(root)
        filenames.append(file)

In [19]:
import pandas as pd

In [20]:
df = pd.DataFrame(folders,columns=['dir'])

In [22]:
df['img'] = filenames

In [35]:
df['rank'] = list(range(len(df)))

In [38]:
df = df.iloc[1:,:]

In [48]:
df['local_path'] = df['dir'] + "/" + df.img

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [50]:
df.reset_index(drop=True,inplace=True)

In [58]:
from shapely.geometry import Point
from PIL import Image
import numpy as np

In [59]:
thumb_side = 128

In [60]:
n = len(df)

In [61]:
side = int(round(np.sqrt(n))) + 4
canvas = Image.new('RGB',(side * thumb_side, side * thumb_side),(50,50,50))
x,y = range(side) * side, np.repeat(range(side),side)
grid_list = pd.DataFrame(x,columns=['x'])
grid_list['y'] = y

In [62]:
point = []
m = len(grid_list.index)
for i in range(m):
    point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))
grid_list['point'] = point
open_grid = list(grid_list.point)

In [63]:
exemplar = Point(int(round(side/2)),int(round(side/2)))
im = Image.open(df.local_path.loc[0])
im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
x = int(exemplar.x) * thumb_side
y = int(exemplar.y) * thumb_side
canvas.paste(im,(x,y))
open_grid.remove(exemplar)

In [64]:
for i in range(1,n):
    im = Image.open(df.local_path.loc[i])
    im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
    closest_open = min(open_grid,key=lambda x: exemplar.distance(x))
    x = int(closest_open.x) * thumb_side
    y = int(closest_open.y) * thumb_side
    canvas.paste(im,(x,y))
    open_grid.remove(closest_open)

In [65]:
canvas.save("/Users/damoncrockett/Desktop/Leo/all.png")